# Fizz Buzz in Tensorflow 
## [@joelgrus](http://twitter.com/joelgrus)
### http://github.com/joelgrus

------------------------


**interviewer:** Welcome, can I get you coffee or anything? Do you need a break?

**me:** No, I've probably had too much coffee already!

**interviewer:** Great, great. And are you OK with writing code on the whiteboard?

**me:** It's the only way I code!

**interviewer:** ...

**me:** That was a joke.

**interviewer:** OK, so are you familiar with "fizz buzz"?

**me:** ...

**interviewer:** Is that a yes or a no?

**me:** It's more of a "I can't believe you're asking me that."

**interviewer:** OK, so I need you to print the numbers from 1 to 100, except that if the number is divisible by 3 print "fizz", if it's divisible by 5 print "buzz", and if it's divisible by 15 print "fizzbuzz".

**me:** I'm familiar with it.

**interviewer:** Great, we find that candidates who can't get this right don't do well here.

**me:** ...

**interviewer:** Here's a marker and an eraser.

**me:** [thinks for a couple of minutes]

**interviewer:** Do you need help getting started?

**me:** No, no, I'm good. So let's start with some standard imports:

In [21]:
import numpy as np
import tensorflow as tf

**interviewer**: Um, you understand the problem is *fizzbuzz*, right?

**me:** Do I ever. So, now let's talk models. I'm thinking a simple multi-layer-perceptron with one hidden layer.

**interviewer:** Perceptron?

**me:** Or neural network, whatever you want to call it. We want the input to be a number, and the output to be the correct "fizzbuzz" representation of that number. In particular, we need to turn each input into a vector of "activations". One simple way would be to convert it to binary.

**interviewer:** Binary?

**me:** Yeah, you know, 0's and 1's? Something like:



In [22]:
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

**interviewer:** [stares at whiteboard for a minute]

**me:** And our output will be a one-hot encoding of the fizzbuzz representation of the number, where the first position indicates "print as-is", the second indicates "fizz", and so on:

In [23]:
def fizz_buzz_encode(i):
    if   i % 15 == 0: return np.array([0, 0, 0, 1])
    elif i % 5  == 0: return np.array([0, 0, 1, 0])
    elif i % 3  == 0: return np.array([0, 1, 0, 0])
    else:             return np.array([1, 0, 0, 0])

**interviewer:** OK, that's probably enough.

**me:** That's enough setup, you're exactly right. Now we need to generate some training data. It would be cheating to use the numbers 1 to 100 in our training data, so let's train it on all the remaining numbers up to 1024:

In [24]:
NUM_DIGITS = 10
trX = np.array([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = np.array([fizz_buzz_encode(i)          for i in range(101, 2 ** NUM_DIGITS)])


**interviewer:** ...

**me:** Now we need to set up our model in tensorflow. Off the top of my head I'm not sure how many hidden units to use, maybe 10?

**interviewer:** ...

**me:** Yeah, possibly 100 is better. We can always change it later.

In [25]:
NUM_HIDDEN = 100


We'll need an input variable with width NUM_DIGITS, and an output variable with width 4:



In [26]:
# https://stackoverflow.com/questions/56226284/why-do-i-get-attributeerror-module-tensorflow-has-no-attribute-placeholder/56230839
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

X = tf.placeholder("float", [None, NUM_DIGITS])
Y = tf.placeholder("float", [None, 4])

**interviewer:** How far are you intending to take this?

**me:** Oh, just two layers deep -- one hidden layer and one output layer. Let's use randomly-initialized weights for our neurons:

In [27]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

w_h = init_weights([NUM_DIGITS, NUM_HIDDEN])
w_o = init_weights([NUM_HIDDEN, 4])

And we're ready to define the model. As I said before, one hidden layer, and let's use, I don't know, ReLU activation:

In [28]:
def model(X, w_h, w_o):
    h = tf.nn.relu(tf.matmul(X, w_h))
    return tf.matmul(h, w_o)

We can use softmax cross-entropy as our cost function and try to minimize it:



In [29]:
py_x = model(X, w_h, w_o)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost)

**interviewer:** ...

**me:** And, of course, the prediction will just be the largest output:

In [30]:
predict_op = tf.argmax(py_x, 1)


**interviewer:** Before you get too far astray, the problem you're supposed to be solving is to generate fizz buzz for the numbers from 1 to 100.

**me:** Oh, great point, the predict_op function will output a number from 0 to 3, but we want a "fizz buzz" output:

In [31]:
def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

**interviewer:** ...

**me:** So now we're ready to train the model. Let's grab a tensorflow session and initialize the variables:

Now let's run, say, 1000 epochs of training?

**interviewer:** ...

**me:** Yeah, maybe that's not enough -- so let's do 10000 just to be safe.


In [32]:
NUM_EPOCHS = 10000

And our training data are sequential, which I don't like, so let's shuffle them each iteration:


And each epoch we'll train in batches of, I don't know, 128 inputs?




In [33]:
BATCH_SIZE = 128


So each training pass looks like



and then we can print the accuracy on the training data, since why not?


interviewer: Are you serious?

me: Yeah, I find it helpful to see how the training accuracy evolves.

interviewer: ...

In [34]:
sess = tf.Session()

with sess.as_default():
    tf.global_variables_initializer().run()
    
    for epoch in range(NUM_EPOCHS):
        p = np.random.permutation(range(len(trX)))
        trX, trY = trX[p], trY[p]
        
        for start in range(0, len(trX), BATCH_SIZE):
            end = start + BATCH_SIZE
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
            
        accuracy = np.mean(np.argmax(trY, axis=1) ==
                           sess.run(predict_op, feed_dict={X: trX, Y: trY}))
        print(epoch, accuracy)

0
7778 1.0
7779 1.0
7780 1.0
7781 1.0
7782 1.0
7783 1.0
7784 1.0
7785 1.0
7786 1.0
7787 1.0
7788 1.0
7789 1.0
7790 1.0
7791 1.0
7792 1.0
7793 1.0
7794 1.0
7795 1.0
7796 1.0
7797 1.0
7798 1.0
7799 1.0
7800 1.0
7801 1.0
7802 1.0
7803 1.0
7804 1.0
7805 1.0
7806 1.0
7807 1.0
7808 1.0
7809 1.0
7810 1.0
7811 1.0
7812 1.0
7813 1.0
7814 1.0
7815 1.0
7816 1.0
7817 1.0
7818 1.0
7819 1.0
7820 1.0
7821 1.0
7822 1.0
7823 1.0
7824 1.0
7825 1.0
7826 1.0
7827 1.0
7828 1.0
7829 1.0
7830 1.0
7831 1.0
7832 1.0
7833 1.0
7834 1.0
7835 1.0
7836 1.0
7837 1.0
7838 1.0
7839 1.0
7840 1.0
7841 1.0
7842 1.0
7843 1.0
7844 1.0
7845 1.0
7846 1.0
7847 1.0
7848 1.0
7849 1.0
7850 1.0
7851 1.0
7852 1.0
7853 1.0
7854 1.0
7855 1.0
7856 1.0
7857 1.0
7858 1.0
7859 1.0
7860 1.0
7861 1.0
7862 1.0
7863 1.0
7864 1.0
7865 1.0
7866 1.0
7867 1.0
7868 1.0
7869 1.0
7870 1.0
7871 1.0
7872 1.0
7873 1.0
7874 1.0
7875 1.0
7876 1.0
7877 1.0
7878 1.0
7879 1.0
7880 1.0
7881 1.0
7882 1.0
7883 1.0
7884 1.0
7885 1.0
7886 1.0
7887 1.0
7888 1.0

**me:** So, once the model has been trained, it's fizz buzz time. Our input should just be the binary encoding of the numbers 1 to 100:

In [35]:
numbers = np.arange(1, 101)
teX = np.transpose(binary_encode(numbers, NUM_DIGITS))

And then our output is just our `fizz_buzz` function applied to the model output:



In [36]:
teY = sess.run(predict_op, feed_dict={X: teX})
output = np.vectorize(fizz_buzz)(numbers, teY)

**interviewer:** ...

**me:** And that should be your fizz buzz!

**interviewer:** Really, that's enough. We'll be in touch.

**me:** In touch, that sounds promising.

**interviewer: ...**

## Postscript

I didn't get the job. It turns out it got some of the outputs wrong! Thanks a lot, machine learning.

In [37]:
actuals = [fizz_buzz(i, fizz_buzz_encode(i).argmax()) for i in numbers]

for i, (predicted, actual) in enumerate(zip(output, actuals)):
    if predicted != actual:
        print("{0} {1} {2}".format(i+1, predicted, actual))

19 fizz 19
21 21 fizz
32 fizz 32
42 42 fizz
81 81 fizz
87 87 fizz


I guess maybe I should have used a deeper network.